In [1]:
import math
import cairo

In [4]:
cairo.HAS_SVG_SURFACE, cairo.HAS_PNG_FUNCTIONS

(1, 1)

In [43]:
def compute_next(s, rules):
    return ''.join(map(lambda x: rules[x], s))

In [161]:
def generate(prod, rules, n):
    if n == 0:
        return prod
    return generate(compute_next(prod, rules), rules, n - 1)

In [132]:
def add_identity_rules(rules):
    keys = set(rules.keys())
    values = set([ch for v in rules.values() for ch in v])
    nonkeys = values.difference(keys)
    return {**rules, **{k: k for k in nonkeys}}

In [158]:
class LSystemCanvas(object):
    
    def __init__(self, width, height, step):
        self.step = step
        self.surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, width, height)
        self.ctx = cairo.Context(self.surface)
        self.ctx.scale(width, height)
        self.ctx.translate(0, 1)
        self.ctx.rotate(math.radians(180))
        
    def draw_line(self):
        self.ctx.line_to(0, self.step)
        self.ctx.translate(0, self.step)        
    
    def rotate(self, degrees):
        self.ctx.rotate(math.radians(degrees))
        
    def save_png(self, filename, color=(0, 0, 0)):
        # add color validation?
        self.ctx.set_source_rgb(*color)
        self.ctx.set_line_width(0.001)
        self.ctx.stroke()
        self.surface.write_to_png(f'{filename}.png')

In [139]:
class QuadGosper(object):
    axiom = '-B'
    rules = add_identity_rules({
        'A': 'AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB-',
        'B': '+AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BB'       
    })

    @classmethod
    def draw(cls, canvas, n=2):
        for item in generate(cls.axiom, cls.rules, n):
            if item in ['A', 'B']:
                canvas.draw_line()
            elif item == '-':
                canvas.rotate(90)
            elif item == '+':
                canvas.rotate(-90)

In [162]:
canvas = LSystemCanvas(2000, 2000, 0.01)
QuadGosper.draw(canvas, 4)
canvas.save_png('quad_gosper')

In [148]:
class Hilbert(object):
    axiom = 'A'
    rules = add_identity_rules({
        'A': '-BF+AFA+FB-',
        'B': '+AF-BFB-FA+'
    })
    
    @classmethod
    def draw(cls, canvas, n=2):
        for item in generate(cls.axiom, cls.rules, n):
            if item == 'F':
                canvas.draw_line()
            elif item == '-':
                canvas.rotate(90)
            elif item == '+':
                canvas.rotate(-90)

In [149]:
canvas = LSystemCanvas(2000, 2000, 0.005)
Hilbert.draw(canvas, 8)
canvas.save_png('hilbert')

In [156]:
class Moore(object):
    axiom = 'LFL+F+LFL'
    rules = add_identity_rules({
        'L': '-RF+LFL+FR-',
        'R': '+LF-RFR-FL+'
    })
    
    @classmethod
    def draw(cls, canvas, n=2):
        for item in generate(cls.axiom, cls.rules, n):
            if item == 'F':
                canvas.draw_line()
            elif item == '-':
                canvas.rotate(-90)
            elif item == '+':
                canvas.rotate(90)

In [ ]:
canvas = LSystemCanvas(2000, 2000, 0.005)
Moore.draw(canvas, 8)
canvas.save_png('moore')

In [ ]:
# END

In [143]:
# class QuadGosper(object):
#     axiom = '-B'
#     rules = add_identity_rules({
#         'A': 'AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB-',
#         'B': '+AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BB'       
#     })
    
#     def __init__(self, ctx):
#         self.ctx = ctx
#         self.ctx.translate(0, 1)
#         self.ctx.rotate(math.radians(180))
        
#     def draw(self, n=2):
#         step = .005
#         for item in generate(QuadGosper.axiom, QuadGosper.rules, n):
#             if item in ['A', 'B']:
#                 self.ctx.line_to(0, step)
#                 self.ctx.translate(0, step)
#             elif item == '-':
#                 self.ctx.rotate(math.radians(90))
#             elif item == '+':
#                 self.ctx.rotate(math.radians(-90))

In [144]:
# generate(QuadGosper.axiom, QuadGosper.rules, 2)

'-+AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB-AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB--+AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BB-+AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BB+AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB-+AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB-+AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BB+AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB--+AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BB+AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BB-AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB--+AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BB+AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB-+AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BB+AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BB-AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB--+AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BBAA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB-+AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB-++AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BB-+AA-B-B+A+AB+A-BB-A-B+ABB-A-BA+A+B-B-A+A+BB-AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B-A+A+BB-+AA-B-B+A+A-B-BA+B+AAB-A+B+AA+B-AB-B

In [145]:
# WIDTH, HEIGHT = 2000, 2000
# surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, WIDTH, HEIGHT)
# ctx = cairo.Context(surface)
# ctx.scale(WIDTH, HEIGHT)

# lsys = QuadGosper(ctx)
# lsys.draw(4)

# ctx.set_source_rgb(0, 0, 0)
# ctx.set_line_width(0.001)
# ctx.stroke()

# surface.write_to_png("example.png")

In [117]:
# class Hilbert(LSystem):
#     axiom = 'A'
#     rules = add_identity_rules({
#         'A': '-BF+AFA+FB-',
#         'B': '+AF-BFB-FA+'
#     })
    
#     def __init__(self, ctx):
#         self.ctx = ctx
#         self.ctx.translate(0, 1)
#         self.ctx.rotate(math.radians(180))
        
#     def draw(self, n=2):
#         step = .005
#         for item in generate(Hilbert.axiom, Hilbert.rules, n):
#             if item == 'F':
#                 self.ctx.line_to(0, step)
#                 self.ctx.translate(0, step)
#             elif item == '-':
#                 self.ctx.rotate(math.radians(90))
#             elif item == '+':
#                 self.ctx.rotate(math.radians(-90))

In [ ]:
# surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, WIDTH, HEIGHT)
# ctx = cairo.Context(surface)

# surface.write_to_png(f'hilbert.png')

In [ ]:
# def draw(filename, lsystem, width=2000, height=2000):
#     surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, WIDTH, HEIGHT)
#     ctx = cairo.Context(surface)
#     ctx.scale(WIDTH, HEIGHT)

#     lsys = Hilbert(ctx)
#     lsys.draw(8)

#     ctx.set_source_rgb(0, 0, 0)
#     ctx.set_line_width(0.001)
#     ctx.stroke()

#     surface.write_to_png(f'{filename}.png')

In [118]:
# WIDTH, HEIGHT = 2000, 2000
# surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, WIDTH, HEIGHT)
# ctx = cairo.Context(surface)
# ctx.scale(WIDTH, HEIGHT)

# lsys = Hilbert(ctx)
# lsys.draw(8)

# ctx.set_source_rgb(0, 0, 0)
# ctx.set_line_width(0.001)
# ctx.stroke()

# surface.write_to_png("example2.png")